This is a baseline version. This version is based on statistics only and doesn't take the user information.
The predict click probability is the average of certain Ad. Nothing hard.

In [20]:
# -*- coding: utf-8 -*-
import zipfile
import numpy as np
import pandas as pd

# load data
dfTrain = pd.read_csv("train.csv")
dfTest = pd.read_csv("test.csv")
dfAd = pd.read_csv("ad.csv")

In [21]:
dfTrain.head()

,label,clickTime,conversionTime,creativeID,userID,positionID,connectionType,telecomsOperator
0,0,170000,NaN,3089,2798058,293,1,1
1,0,170000,NaN,1259,463234,6161,1,2
2,0,170000,NaN,4465,1857485,7434,4,1
3,0,170000,NaN,1004,2038823,977,1,1
4,0,170000,NaN,1887,2015141,3688,1,1


In [22]:
dfTest.head()

,instanceID,label,clickTime,creativeID,userID,positionID,connectionType,telecomsOperator
0,1,-1,310000,3745,1164848,3451,1,3
1,2,-1,310000,2284,2127247,1613,1,3
2,3,-1,310000,1456,2769125,5510,2,1
3,4,-1,310000,4565,9762,4113,2,3
4,5,-1,310000,49,2513636,3615,1,3


In [23]:
dfAd.head()

,creativeID,adID,camgaignID,advertiserID,appID,appPlatform
0,4079,2318,147,80,14,2
1,4565,3593,632,3,465,1
2,3170,1593,205,54,389,1
3,6566,2390,205,54,389,1
4,5187,411,564,3,465,1


In [24]:
# process data
dfTrain = pd.merge(dfTrain, dfAd, on="creativeID")
dfTest = pd.merge(dfTest, dfAd, on="creativeID")
y_train = dfTrain["label"].values

In [25]:
dfTrain.head()

,label,clickTime,conversionTime,creativeID,userID,positionID,connectionType,telecomsOperator,adID,camgaignID,advertiserID,appID,appPlatform
0,0,170000,NaN,3089,2798058,293,1,1,1321,83,10,434,1
1,0,170001,NaN,3089,195578,3659,0,2,1321,83,10,434,1
2,0,170014,NaN,3089,1462213,3659,0,3,1321,83,10,434,1
3,0,170030,NaN,3089,1985880,5581,1,1,1321,83,10,434,1
4,0,170047,NaN,3089,2152167,5581,1,1,1321,83,10,434,1


In [26]:
dfTest.head()

,instanceID,label,clickTime,creativeID,userID,positionID,connectionType,telecomsOperator,adID,camgaignID,advertiserID,appID,appPlatform
0,1,-1,310000,3745,1164848,3451,1,3,1166,430,80,14,2
1,9,-1,310000,3745,1113275,3347,1,2,1166,430,80,14,2
2,43,-1,310000,3745,1215329,3347,1,1,1166,430,80,14,2
3,83,-1,310000,3745,2077956,3347,1,2,1166,430,80,14,2
4,108,-1,310000,3745,2343346,7422,1,1,1166,430,80,14,2


In [27]:
# model building
key = "appID"
dfCvr = dfTrain.groupby(key).apply(lambda df: np.mean(df["label"])).reset_index()
dfCvr.columns = [key, "avg_cvr"]
dfTest = pd.merge(dfTest, dfCvr, how="left", on=key)
dfTest["avg_cvr"].fillna(np.mean(dfTrain["label"]), inplace=True)
proba_test = dfTest["avg_cvr"].values

# submission
df = pd.DataFrame({"instanceID": dfTest["instanceID"].values, "proba": proba_test})
df.sort_values("instanceID", inplace=True)

df.head()

,instanceID,proba
0,1,0.002515
1238,2,0.002515
1887,3,0.020328
12634,4,0.020328
35772,5,0.002515
